Analyse de l'image 

In [2]:
!pip install webcolors==1.3

Defaulting to user installation because normal site-packages is not writeable


In [1]:
exif_data_interested = [
    "DateTimeOriginal",
    "Make",
    "Model",
    "Orientation",
    "Copyright",
    "DateTime",
    "DateTimeDigitized"
]

In [2]:
import os
import json
from PIL import Image, ImageColor
from PIL.ExifTags import TAGS
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
import webcolors
import math


def getDominatedColors(imgfile):
    # Parcourir les fichiers dans le répertoire d'images
    print(imgfile)
    numarray = np.array(imgfile.getdata(), np.uint8)
    colors = []
    if len(numarray.shape) == 2:
        clusters = KMeans(n_clusters=2, n_init=2)
        clusters.fit(numarray)
        for i in range(len(clusters.cluster_centers_)):
            colors.append(
                "#%02x%02x%02x"
                % (
                    math.ceil(clusters.cluster_centers_[i][0]),
                    math.ceil(clusters.cluster_centers_[i][1]),
                    math.ceil(clusters.cluster_centers_[i][2]),
                )
            )
    return colors

# https://stackoverflow.com/questions/70967119/webcolors-has-no-attribute-css3-hex-to-names
def get_closest_color(rgb_triplet):
    min_colours = {}
    for key, name in webcolors.css3_hex_to_names.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - rgb_triplet[0]) ** 2
        gd = (g_c - rgb_triplet[1]) ** 2
        bd = (b_c - rgb_triplet[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, bytes):
            return obj.decode('utf-8', 'ignore')
        return super().default(obj)

metadata = []

# Parcourir les fichiers dans le répertoire d'images
for filename in os.listdir("images"):
    if filename.endswith('.jpg') or filename.endswith('.JPG') or filename.endswith('.jpeg') or filename.endswith('.png'):
        with Image.open(os.path.join("images", filename)) as img:
            json_path = os.path.join("metadata", filename.replace(".jpg", ".json").replace(".JPG", ".json").replace(".jpeg", ".json").replace(".png", ".json"))
            if os.path.exists(json_path):
                with open(json_path, 'r') as f:
                    existing_metadata = json.load(f)
            # else:
            #     existing_metadata = []

            if 'dominated_colors_name' in existing_metadata.keys() :
                print("Already treated")
                pass

            exif_data = img.getexif()
            exif = {}
            if exif_data:
                for tag, value in exif_data.items():
                    decoded_tag = TAGS.get(tag, tag)
                    if decoded_tag in exif_data_interested:
                        exif[decoded_tag] = value

            colors_dominated = getDominatedColors(img)
            colors_dominated_name = []

            print(colors_dominated)

            for color in colors_dominated:
                colors_dominated_name.append(get_closest_color(ImageColor.getcolor(color, "RGB")))



            # Add the new metadata to the list
            # print(existing_metadata)

            existing_metadata['filename']= filename
            existing_metadata['width']= img.width
            existing_metadata['height']= img.height
            existing_metadata['format']= img.format
            existing_metadata['mode']= img.mode
            existing_metadata['dominated_colors']= colors_dominated
            existing_metadata['dominated_colors_name']= colors_dominated_name
            existing_metadata['exif']= exif


            # Save the metadata to a JSON file
            with open(json_path, 'w') as f:
                json.dump(existing_metadata, f, cls=CustomEncoder)

Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x800 at 0x7FB5D1AE8760>
['#13240d', '#a98e6c']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2448x2994 at 0x7FB5CF643700>
['#49642a', '#92a76f']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2400x3200 at 0x7FB5CF643D60>
['#a39c98', '#564f47']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1835x2500 at 0x7FB5CF643640>
['#d9d0c5', '#807d4b']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1169x973 at 0x7FB5CF643700>
['#4a5727', '#acbc74']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=907x738 at 0x7FB5CF6434C0>
['#4f6a31', '#9ba785']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1707x1476 at 0x7FB5CF6485B0>
['#a7a166', '#412524']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1300x2913 at 0x7FB5CF648220>
['#e6e2da', '#5b4e38']
Already treated
<PIL.

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 179, in where
KeyboardInterrupt: 


['#837545', '#383115']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1378x958 at 0x7FB5CF64A0D0>
['#211e12', '#93917f']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=414x779 at 0x7FB5CF64AAC0>
['#425c66', '#abafab']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=389x589 at 0x7FB5CF64A160>
['#a3a786', '#303d29']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2272x1704 at 0x7FB5CF64AF40>
['#7a6d5e', '#bea9af']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=699x600 at 0x7FB5CF64A1C0>
['#606038', '#c5c3b4']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x7FB5CF64A790>
['#a69a8d', '#564a38']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1999x1366 at 0x7FB5CF6B5070>
['#bec3af', '#444421']
Already treated
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1373x829 at 0x7FB5CF6B5790>
['#f0f0f0', '#494949']
Alr

KeyboardInterrupt: 